The following code attempts to classify land and predict the expansion of land.

This starts by reading in the a bunch of different TIF files and classifying them into various classifications.

In [1]:
import matplotlib
import numpy as np
from numpy import save
import matplotlib.pyplot as plt
import boto3
from botocore import UNSIGNED
from botocore.client import Config
import os
import cv2
import math
import glob

import zipfile
import tarfile
import shutil
from PIL import Image
from osgeo import gdal
import sys
import webcolors

import pickle
import random

import json
#import rsgislib
#from rsgislib import imageutils

Deletes all the RED.TIF and NIR.TIF data used in the NDVI calculation.

In [53]:
def deleteData(location):
    listofDir = os.listdir(location)
    for i in listofDir:
        bandsLocation = location + "/" + i
        listofBands = os.listdir(bandsLocation)
        fileLocationInList = 0
        for file in listofBands:   
            try:
                if "Blue.TIF" in file:
                    band3 = bandsLocation + "/Blue.TIF"
                    os.remove(band3)
            except:
                pass 
            
            
            try:
                if "Green.TIF" in file:
                    band3 = bandsLocation + "/Green.TIF"
                    os.remove(band3)
            except:
                pass 
            
            
            
            try:
                if "RED.TIF" == file:
                    band2 = bandsLocation + "/Red.TIF"
                    os.remove(band2)
            except:
                pass

            try:
                if "NIR.TIF" in file:
                    band3 = bandsLocation + "/NIR.TIF"
                    os.remove(band3)
            except:
                pass 
            
            
            
            



In [54]:
deleteData("/media/russell/775C-44EC/LandsatExport/Victoria/Landsat4-8")

This program requires several different landsat images over a course of several years. I recommend having 
1 image/year. Make sure the image has little to no cloud coverage over the study area. I recommend gathering the data from GloVis. As it is fast and easy to gather data from this source.

In [2]:
def unpackTarGZ(archives, extract_path):
    extract_path = extract_path+"/dir1"
    dirCount = 1
    for filename in archives:
        os.mkdir(extract_path)
        try:
            shutil.unpack_archive(filename, extract_path)
        except:
            pass
        print(extract_path)
        res = extract_path.rsplit('/', 1)
        dirCount += 1
        extract_path = res[0] + "/dir" + str(dirCount)

This next bit assumes you download the data in chronological order, it may need to be changed at a later point.

In [ ]:
path = "/media/russell/775C-44EC/Landsat4-7/Victoria"
pathExport = "/media/russell/775C-44EC/LandsatExport/Victoria/Landsat4-8"
uncompressionList = os.listdir(path)
uncompressionList=[path+"/" +s for s in uncompressionList]
uncompressionList.sort()

Landsat Data Can be very large so it needs to be transformed to allow for the quicker analysis of data.

In [51]:
def reshapeImage(Blue,Green,Red,nir,Path):
    BlueBand = "gdalwarp -t_srs EPSG:3857 -te -13757556.7544 6173245.4969 -13720790.5438 6192431.1910 " + Blue + " " +Path + "/Blue.TIF"
    GreenBand = "gdalwarp -t_srs EPSG:3857 -te -13757556.7544 6173245.4969 -13720790.5438 6192431.1910 " + Green + " " +Path + "/Green.TIF"
    RedBand = "gdalwarp -t_srs EPSG:3857 -te -13757556.7544 6173245.4969 -13720790.5438 6192431.1910 " + Red + " " +Path + "/Red.TIF"
    #trueColour= "gdalbuildvrt -separate " + Path + "/RGB.vrt " + Red + " " + Green + " " + Blue
    if nir!=False:
            nirBand = "gdalwarp -t_srs EPSG:3857 -te -13757556.7544 6173245.4969 -13720790.5438 6192431.1910 " + nir + " " +Path + "/NIR.TIF"
            os.system(nirBand)

    os.system(BlueBand)
    os.system(GreenBand)
    os.system(RedBand)
    
   

Landsat 4-7  
    Red Band:   3
    Green Band: 2
    Blue Band:  1
    
Landsat 8:
    Red Band:   4
    Green Band: 3
    Blue Band:  2  


In [55]:
dataLocation = "/media/russell/775C-44EC/LandsatExport/Victoria/Landsat4-8"
listofDir = os.listdir(dataLocation)
for i in listofDir:
    bandsLocation = dataLocation + "/" + i
    listofBands = os.listdir(bandsLocation)
    fileLocationInList = 0
    nir=False   
    for file in listofBands:
        if "LC08" in file:
            if "B5.TIF" in file:
                nir = bandsLocation + "/" + listofBands[fileLocationInList] 
            if "B4.TIF" in file:
                red = bandsLocation + "/" + listofBands[fileLocationInList]
            if "B3.TIF" in file:
                green = bandsLocation + "/" + listofBands[fileLocationInList]
            if "B2.TIF" in file:
                blue = bandsLocation + "/" + listofBands[fileLocationInList]
          
        else:
            if "B4.TIF" in file:
                nir = bandsLocation + "/" + listofBands[fileLocationInList] 
            if "B3.TIF" in file:
                red = bandsLocation + "/" + listofBands[fileLocationInList]
            if "B2.TIF" in file:
                green = bandsLocation + "/" + listofBands[fileLocationInList]
            if "B1.TIF" in file:
                blue = bandsLocation + "/" + listofBands[fileLocationInList]
    
        fileLocationInList += 1
    reshapeImage(blue,green,red,nir,bandsLocation)
    #print("Created Image")

In [25]:
#NDVI Band L4-7
def reviewReds(filepath):
    redGDAL = gdal.Open(filepath+"/RED.TIF")
    nirGDAL = gdal.Open(filepath+"/NIR.TIF")
    redGDAL = np.array(redGDAL.GetRasterBand(1).ReadAsArray())
    nirGDAL = np.array(nirGDAL.GetRasterBand(1).ReadAsArray())
    numrows = redGDAL.shape[0]
    numcols = nirGDAL.shape[1]
    landClassification = np.empty((numrows,numcols),dtype=object)
    img = np.zeros((numrows,numcols,3),np.uint8)
    for x in range(numrows):
        for y in range(numcols):
            redV = redGDAL[x,y]
            nirV = nirGDAL[x,y]
            numtpa = (int(nirV) - int(redV))
            dem = (int(nirV) + int(redV))
            if(dem!=0):
                NDVI = numtpa/dem
                if NDVI<-0.02:
                    img[x, y] = [255, 0, 0]
                    landClassification[x, y] = "Water"
                elif NDVI>=0.15:
                    img[x, y] = [0, 128, 0]
                    landClassification[x, y] = "Heavy Vegetation"

                else:
                    img[x,y] = [60,60,60]
                    landClassification[x, y] = "Urban"
    
    save(filepath+"/data.npy",landClassification)
    cv2.imwrite(filepath+"/classification.png",img)
    print("Finished Creating Numpy")

This next section creates create the inital set of numpy files.
This contains the results of the NDVI calculation. 

In [27]:
location="/media/russell/775C-44EC/LandsatExport/Victoria/Landsat4-8"
listofDir = os.listdir(location)
for i in listofDir:
        bandsLocation = location + "/" + i
        reviewReds(bandsLocation)
        #reviewRedsL8(bandsLocation)
        


Finished Creating Numpy
Finished Creating Numpy
Finished Creating Numpy
Finished Creating Numpy
Finished Creating Numpy
Finished Creating Numpy
Finished Creating Numpy
Finished Creating Numpy
Finished Creating Numpy
Finished Creating Numpy
Finished Creating Numpy
Finished Creating Numpy
Finished Creating Numpy
Finished Creating Numpy
Finished Creating Numpy
Finished Creating Numpy
Finished Creating Numpy
Finished Creating Numpy
Finished Creating Numpy


In [41]:
def checkRGB(filepath):
    RedGDAL = gdal.Open("/media/russell/775C-44EC/LandsatExport/Victoria/Landsat4-8/dir1/Red.TIF")
    #BlueGDAL = gdal.Open(filepath+"/Blue.TIF")
    BlueGDAL = gdal.Open("/media/russell/775C-44EC/LandsatExport/Victoria/Landsat4-8/dir1/Blue.TIF")
    GreenGDAL = gdal.Open("/media/russell/775C-44EC/LandsatExport/Victoria/Landsat4-8/dir1/Green.TIF")
    
    RedGDAL = np.array(RedGDAL.GetRasterBand(1).ReadAsArray())
    BlueGDAL= np.array(BlueGDAL.GetRasterBand(1).ReadAsArray())
    GreenGDAL = np.array(GreenGDAL.GetRasterBand(1).ReadAsArray())
    numrows = RedGDAL.shape[0]
    numcols = RedGDAL.shape[1]
    img = np.zeros((numrows,numcols,3),np.uint8)
    landClassification = np.empty((numrows,numcols),dtype=object)
    for x in range(0,numrows):
        for y in range(0,numcols):
            redPixel = RedGDAL[x,y]
            bluePixel = BlueGDAL[x,y]
            greenPixel = GreenGDAL[x,y]
            img[x,y]=[bluePixel,greenPixel,redPixel]
            
              
    print(filepath+"/classification.png")
    cv2.imwrite(filepath+"/classification.png",img)        
        
checkRGB("/media/russell/775C-44EC/LandsatExport/Victoria")    

/media/russell/775C-44EC/LandsatExport/Victoria/classification.png


This next bit goes all over the recently created pixels and creates models. These models will take all the surroudning pixels and change these pixels based off a set of rules created. 

The NDVI calculation is a good way at generally classifiying land, there are better ways but for the time constraints of this project this seemed to be the best method. 


The models take all the surrounding pixels. These models will be stored into a numpy file. The order of the models is 
[CurrentPixel,NorthPixel,NorthEastPixel,EastPixel,SouthEastPixel,SouthPixel,SouthWestPixel,WestPixel,NorthWest]


The NDVI isn't always accurate and sometimes can misclassify Water, and Urban. The Final image will be an average of all the images over the time series. 

 


In [16]:
def analysisData(location):
    listofDir = os.listdir(location)
    count = 0
    data = np.load("/media/russell/775C-44EC/Models/Victoria/data18.npy",allow_pickle=True)
    largeRow = data.shape[0]
    largeCol = data.shape[1]
    UrbanCount = np.zeros((largeRow, largeCol))
    TreeCount = np.zeros((largeRow,largeCol))
    WaterCount = np.zeros((largeRow,largeCol))
    landClassification2 = np.empty((largeRow,largeCol),dtype=object)
    for i in listofDir:
        npyBand = location + "/" +i
        listofBands = os.listdir(npyBand)
        for file in listofBands:
            if ".npy" in file:
                data = np.load(npyBand+"/"+file,allow_pickle=True)
                numrows = data.shape[0]
                numcols = data.shape[1]
                surroundingPixels = np.empty((numrows,numcols),dtype=object)
                img = np.zeros((numrows,numcols,3),np.uint8)
                for x in range(0,numrows):
                    for y in range(0,numcols):
                        landClassification = data[x,y]
                        northPixel = "Unclassified"
                        northEastPixel = "Unclassified"
                        
                        eastPixel = "Unclassified"
                        southEastPixel = "Unclassified"
                        
                        southPixel = "Unclassified"
                        southWestPixel = "Unclassified"
                        
                        westPixel = "Unclassified"
                        northWestPixel = "Unclassified"
                        
                        allLocations = []   
                        if y!=0:
                            northPixel = data[x,y-1]
                        if y!=0 and x!=numrows-1:
                            northEastPixel=data[x+1,y-1]
                        if x!= numrows-1:
                            eastPixel=data[x+1,y]
                        if  x!=numrows-1  and y!=numcols-1:
                            southEastPixel = data[x+1,y+1]
                        if y!=numcols-1:
                            southPixel = data[x,y+1]
                        if x!=0 and y!=numcols-1 :
                            southWestPixel = data[x-1,y+1]
                        if x!=0:
                            westPixel = data[x-1,y]
                        if x!=0 and y!=0:
                            northWestPixel = data[x-1,y-1]
                        
                        allLocations.append(landClassification)    
                        allLocations.append(northPixel)
                        allLocations.append(northEastPixel)
                        allLocations.append(eastPixel)
                        allLocations.append(southEastPixel)
                        allLocations.append(southPixel)
                        allLocations.append(southWestPixel)
                        allLocations.append(westPixel)
                        allLocations.append(northWestPixel)
                        
                        
                        waterCountComp = allLocations.count("Water")
                        urbanCountComp = allLocations.count("Urban")
                        treeCountComp = allLocations.count("Heavy Vegetation")
                        
                        if urbanCountComp>=4 and allLocations[0]!="Urban":
                            allLocations[0]="Urban"
                            data[x,y] = "Urban"
                            #print("Changing to Urban")
                            
                        if treeCountComp>=7 and allLocations[0]!="Heavy Vegetation":
                            allLocations[0]="Heavy Vegetation"
                            data[x,y] = "Heavy Vegetation"
                            #print("Changing to Vegetation")
                            
                        if waterCountComp>6 and allLocations[0]!="Water":
                            allLocations[0]="Water"
                            data[x,y]="Water"
                            #print("Changing to Water")
                            
                        surroundingPixels[x,y] = allLocations
                        if allLocations[0]=="Water":
                            img[x, y] = [255, 0, 0]
                            try:
                                WaterCount[x,y]+=1
                            except:
                                pass
                        
                        if allLocations[0]=="Urban":
                            img[x, y] = [60, 60, 60]
                            try:
                                UrbanCount[x,y]+=1
                            except:
                                pass
                        if allLocations[0]=="Heavy Vegetation":
                            img[x, y] = [0, 128, 0]
                            try:
                                TreeCount[x,y]+=1
                            except:
                                pass
                        
                    
                        
                        
                        
                modelsPath = "/media/russell/775C-44EC/Models/Victoria/"
                save(modelsPath+"/data"+str(count)+".npy",surroundingPixels)
                cv2.imwrite(modelsPath+"/classification"+str(count)+".png",img) 
                print("Created Model")
                print("Created Map")
                count+=1
        
                        
    img = np.zeros((numrows,numcols,3),np.uint8)                   
    for x in range(largeRow):
        for y in range(largeCol):
            wC = WaterCount[x,y]
            uC = UrbanCount[x,y]
            tC = TreeCount[x,y]                
            if wC> uC and wC>tC:
                img[x,y]=[255,0,0]
                landClassification2[x,y] = "Water"
                
            elif uC>wC and uC>tC:
                img[x,y]=[60,60,60]
                landClassification2[x,y] = "Urban"
                
            elif tC>wC and tC>uC:
                img[x,y]=[0,128,0]
                landClassification2[x,y] ="Heavy Vegetation"
                
            else:
                img[x,y]=[255,0,0]
                landClassification2[x,y] = "Urban"
            
    
    print("Create Final Image")
    modelsPath = "/media/russell/775C-44EC/Models/Victoria/"            
    cv2.imwrite(modelsPath+"/finalImage.png",img)
    save(modelsPath+"/finalNPY.npy",landClassification2)        
            

In [40]:
patterenLocation = "/media/russell/775C-44EC/LandsatExport/Victoria/Landsat4-8"
analysisData(patterenLocation)

Created Model
Created Map
Created Model
Created Map
Created Model
Created Map
Created Model
Created Map
Created Model
Created Map
Created Model
Created Map
Created Model
Created Map
Created Model
Created Map
Created Model
Created Map
Created Model
Created Map
Created Model
Created Map
Created Model
Created Map
Created Model
Created Map
Created Model
Created Map
Created Model
Created Map
Created Model
Created Map
Created Model
Created Map
Created Model
Created Map
Created Model
Created Map
Create Final Image


In [4]:
def checkIfNew(compare,currentModel,dict):
    if compare not in dict:
        dict[compare] = {}
    try:
        dict[compare][currentModel] += 1
    except:
        dict[compare][currentModel] = 1
    return dict         

In [5]:
def stringCreator(listofData):
        mod=[]
        mod.extend(listofData)
        modS = ""
        try:
            modS = modS.join(mod)
            return modS
        except:
            return False  

In [6]:
def xstr(s):
    if s is None:
        return ''
    return s

In [7]:
def compareHowModelsChange(pixelModels):
    #listOfModels = os.listdir(pixelModels)
    #listOfModels.sort()

    listOfModels = list(filter(os.path.isfile, glob.glob(pixelModels + "*")))
    listOfModels.sort(key=lambda x: os.path.getmtime(x))
    #print(listOfModels)
    #Used for all Models
    modelFrequency = []

    previousModelD = {}
    previousModels = []

    modelTransformation = {}
    modelStagnant = {}
    modAll = {}

    notFirstModel = 0

    #Read in List of Models
    for i in listOfModels:
         if (".npy" in i and "finalNPY" not in i):
                data = np.load(i,allow_pickle=True)
                row = data.shape[0]
                col = data.shape[1]
                if notFirstModel==0:
                    notFirstModel+=1
                    for x in range(row):
                        for y in range(col):
                            previousModels.append(data[x,y])
                    previousModelNP = np.asarray(previousModels, dtype=object)
                else:
                    countForPreviousModel = 0
                    previousModels=[]
                    for x in range(row):
                        for y in range(col):
                            currentModel = data[x,y]
                            compare=["","","","","","","","",""]
                            try:
                                if countForPreviousModel<len(previousModelNP):
                                        compare = previousModelNP[countForPreviousModel].tolist()
                                        countForPreviousModel+=1
                            except:
                                pass
                                #  print("Always Reached")
                               # break
                            
                            
                            compare = [xstr(s) + "," for s in compare]
                            currentModel = [xstr(s) + "," for s in currentModel]
                            compareS = stringCreator(compare)
                            currentModelS = stringCreator(currentModel)
                                                        
                            if currentModelS!= False and compareS!=False:
                                #print(compareS)
                                #print("\n")
                                #print(currentModelS)
                                modAll = checkIfNew(compareS,currentModelS,modAll)
                                
                            

    print("Writing")
    with open(pixelModels+"/transPickle.p",'wb') as fp:
        pickle.dump(modelTransformation, fp, protocol=pickle.HIGHEST_PROTOCOL)

    with open(pixelModels+"/stagPickle.p","wb") as fp:
        pickle.dump(modelStagnant,fp,protocol=pickle.HIGHEST_PROTOCOL)

    with open(pixelModels+"/mod.p","wb") as fp:
        pickle.dump(modAll,fp,protocol=pickle.HIGHEST_PROTOCOL)
    
    

In [18]:
patternLocation = "/media/russell/775C-44EC/Models/Victoria/"
compareHowModelsChange(patternLocation)

Writing


This next takes in the new Models, and attempts to create a map to 
create a visulization of map.

In [9]:
def convertNumpy(numpy):
    return numpy.tostring()

In [10]:
def numpyToString(previous):
    previous = ",,".join(previous)
    previous+=",,"
    previous = previous.replace(",,",",")
    return previous
                

The following program implements the changes over the next several years given certain rule sets

The Rules are: 

Basic Rules:
1. Once a place has been classified as Urban.
2. Water can not be transformed into Urban.
3. Water can not be transformed into Vegetation.
4. The only places that can change are areas that are Classified as Vegetation



Advanced Rules:





In [41]:
npyBand  = "/media/russell/775C-44EC/Models/Victoria/finalNPY.npy"
allModel = "/media/russell/775C-44EC/Models/Victoria/mod.p"

data = np.load(npyBand,allow_pickle=True)
numrows = data.shape[0]
numcols = data.shape[1]
pickleMod = open(allModel,"rb")
mod = pickle.load(pickleMod)
oldChecks = {}
 
    
for k,v in mod.items():
    previous = k.split(",")[0]
    if previous == "Heavy Vegetation":
        for k2,v2 in v.items():
            previous2 = k2.split(",")[0]
            #Add k2 split instead of previous
            urbanCount = k2.count("Urban") 
            tree = "Heavy Vegetation"+ str(k2.count("Heavy Vegetation"))
            water = "Water"+str(k2.count("Water"))           
            urban = "Urban"+str(k2.count("Urban"))
            
            #if urbanCount<2 and previous2 == "Urban" and urbanCount!=0:
            #    urban = "Road"+str(k2.count("Road"))
            #    previous2 = "Road"
            
            
            key= tree+water+urban
                     
            if previous2 not in oldChecks:
                oldChecks[previous2] = {}
            try:
                oldChecks[previous2][key] += v2
            except:
                oldChecks[previous2][key] = v2
            
        
pixelModels = "/media/russell/775C-44EC/Models/Victoria/"                        
with open(pixelModels+"/lookUp.p","wb") as fp:
        pickle.dump(oldChecks,fp,protocol=pickle.HIGHEST_PROTOCOL)    

f=open("/media/russell/775C-44EC/Models/Victoria/checkOutput.txt","w")
for k,v in oldChecks.items():
    for k2,v2 in v.items():
        s=str(k)+"   "+ str(k2) + " " +str(v2)+"\n"
        #print(s)
        b=f.write(s)       
        

In [42]:
def growthPrediction(npyBand,allModels):
    print("Start")
    data = np.load(npyBand,allow_pickle=True)
    numrows = data.shape[0]
    numcols = data.shape[1]
    pickleMod = open(allModels,"rb")
    mod = pickle.load(pickleMod)
    
    #As the samples gathered
    for i in range(0,10):       
        
        data = np.load(npyBand,allow_pickle=True)
        newData = np.empty((numrows,numcols),dtype=object)       
        dataPixelMap = np.zeros((numrows,numcols,3),dtype=np.uint8)     
        for x in range(0,numrows):
            for y in range(0,numcols):
                surroundingPixels = []                
                northPixel="Unclassifed"
                northEastPixel ="Unclassifed"
                eastPixel= "Unclassifed"
                southEastPixel = "Unclassifed"
                southPixel="Unclassifed"
                southWestPixel = "Unclassifed"
                westPixel= "Unclassifed"
                northWestPixel = "Unclassifed" 
                displayValue = ""
                
                landClassification = data[x,y]
                if y!= 0:
                    northPixel=data[x,y-1]
                if y!=0 and x!= numrows - 1:
                    northEastPixel=data[x+1,y-1]
                if x != numrows-1:
                    eastPixel=data[x+1,y]
                if  x!=numrows-1  and y!=numcols-1:
                    southEastPixel = data[x+1,y+1]
                if y!=numcols-1:
                    southPixel = data[x,y+1]
                if x!=0 and y!=numcols-1:
                    southWestPixel = data[x-1,y+1]
                if x!=0:
                    westPixel = data[x-1,y]
                if x!=0 and y!=0:
                    northWestPixel = data[x-1,y-1]
                
                surroundingPixels.append(landClassification)
                surroundingPixels.append(northPixel)
                surroundingPixels.append(northEastPixel)
                surroundingPixels.append(eastPixel)
                surroundingPixels.append(southEastPixel)
                surroundingPixels.append(southPixel)
                surroundingPixels.append(southWestPixel )
                surroundingPixels.append(westPixel)
                surroundingPixels.append(northWestPixel)
                
                
                urbanCount = surroundingPixels.count("Urban")
                #if urbanCount<2: 
                 #       for n, l in enumerate(surroundingPixels):
                  #           if l == "Urban":
                   #                   surroundingPixels[n] = "Road"
                
                
                
                urString = "Urban"
                
                if landClassification == "Heavy Vegetation":
                        tree = "Heavy Vegetation"+ str(surroundingPixels.count("Heavy Vegetation"))
                        water = "Water"+str(surroundingPixels.count("Water"))
                        urban = "Urban"+str(surroundingPixels.count("Urban"))                           
                        
                       # if urbanCount<2:
                       #     urban = "Road"+str(surroundingPixels.count("Road")) 
                       #     urString = "Road"
                            
                        key = tree+water+urban  
                        value1 = 1
                        value2 = 1
                        try:
                            #print(key)
                            value1 = mod["Heavy Vegetation"][key]
                        except:
                            pass
                        
                        try:
                            key
                            value2 = mod[urString][key]
                        except:
                            pass
                
                        #The following are were the rules are located
                        total = value1+value2
                        number  = random.randrange(0,total)        
                        
                        if number<value1:
                            displayValue = "Heavy Vegetation"

                        elif urString == "Urban":
                            displayValue = "PredictUrban"
                        
                        elif urString == "Road":
                            displayValue = "PredictRoad"
                        
                               
                elif landClassification == "Urban":
                        displayValue = "Urban"
                
                elif landClassification == "Water":
                        displayValue = "Water"
                    
                        
                        
                        
                    
                           
                else:
                    displayValue = landClassification
                 
               
            
                if displayValue == "Urban":
                    newData[x,y] = displayValue
                    dataPixelMap[x,y] = [60,60,60]
                
                elif displayValue == "Heavy Vegetation":
                    newData[x,y] = displayValue
                    dataPixelMap[x,y] = [0,128,0]
                     
                elif displayValue == "Water":
                    newData[x,y] = displayValue
                    dataPixelMap[x,y] = [255,0,0]
                
                elif displayValue == "Road":
                    newData[x,y] = displayValue
                    dataPixelMap[x,y] = [255,255,255]  
                
                elif displayValue == "PredictUrban":
                    dataPixelMap[x,y] = [0,0,255]
                    newData[x,y] = "Urban" 
                     
                elif displayValue == "PredictRoad":
                    dataPixelMap[x,y] = [45,98,100]
                    newData[x,y] = "Road"
                    
                        
                else:
                    #print("Reached")
                    dataPixelMap[x,y]=[0,128,0]
                    newData[x,y] = displayValue 
                  
     
        save(npyBand,newData)
        print("Reached")
        print(i)
        
        if i == 0:
            print("Finished Creating Year 2")
            cv2.imwrite("/media/russell/775C-44EC/Models/Victoria/2years.png",dataPixelMap)

        if i == 4:
            print("Finished Creating Year 5")
            cv2.imwrite("/media/russell/775C-44EC/Models/Victoria/5years.png",dataPixelMap)

        if i == 9:
            print("Finished Creating Year 10")
            cv2.imwrite("/media/russell/775C-44EC/Models/Victoria/10years.png",dataPixelMap)

        
            
            
            
            
            
            
            
            
    
    

In [43]:
npyBand  = "/media/russell/775C-44EC/Models/Victoria/finalNPY.npy"
allModel = "/media/russell/775C-44EC/Models/Victoria/lookUp.p"
growthPrediction(npyBand,allModel)

Start
Reached
0
Finished Creating Year 2
Reached
1
Reached
2
Reached
3
Reached
4
Finished Creating Year 5
Reached
5
Reached
6
Reached
7
Reached
8
Reached
9
Finished Creating Year 10
